In [1]:
import pandas as pd 
import numpy as np

In [2]:
data_train=pd.read_csv('data_train.txt',index_col=[0],header=0)
data_valid=pd.read_csv('data_valid.txt',index_col=[0],header=0)

C:\Users\oyrx\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def is_noise(data_train,data_test):
    #data_train 需要标记出noise的数据，以便建模
    data_train['is_noise']=0
    data_train['is_noise'][data_train['non_ch']==1]=1
    data_train['is_noise'][data_train['length_all']<=5]=1
    data_train['is_noise'][data_train['auto']==1]=1
    data_train['is_noise'][data_train['interaction']==1]=1
    data_train['is_noise'][data_train['vote']==1]=1
    data_train['is_noise'][data_train['ad']==1]=1
    
    #data_test 需要标记出noise的数据，以便预测
    data_test['is_noise']=0
    data_test['is_noise'][data_test['non_ch']==1]=1
    data_test['is_noise'][data_test['length_all']<=5]=1
    data_test['is_noise'][data_test['auto']==1]=1
    data_test['is_noise'][data_test['interaction']==1]=1
    data_test['is_noise'][data_test['vote']==1]=1
    data_test['is_noise'][data_test['ad']==1]=1
    
    #data_test的用户数据需要从data_train中获取
    user_history=data_train[['user_id','number_in_train','forward_max','comment_max','like_max','forward_min','comment_min','like_min','forward_mean','comment_mean','like_mean','forward_more_ave_pr','comment_more_ave_pr','like_more_ave_pr','max_f/l','max_c/l','min_f/l','min_c/l','mean_f/l','mean_c/l']]
    user_history=user_history.drop_duplicates()
    data_test=pd.merge(data_test,user_history,on='user_id',how='left',indicator=False)
    data_test=data_test.fillna(-1)  #将缺失用户的数值标记为-1，为了与互动数为0的用户区分开来
    #如果data_test中出现新用户，标记为noise值
    data_test['is_noise'][data_test['number_in_train']==-1]=1
    #如果data_test中的旧用户的互动数为0，标记为noise值
    data_test['is_noise'][(data_test['forward_max']+data_test['comment_max']+data_test['like_max'])==0]=1
    return data_train,data_test

In [4]:
data_train,data_valid=is_noise(data_train,data_valid)

C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher

In [5]:
data_train.to_csv('data_train.txt',header=True)
data_valid.to_csv('data_valid.txt',header=True)

In [6]:
data_valid[['text','repost','comments','likes']][data_valid['is_noise']==1].head()

,text,repost,comments,likes
9,【*Get started with functional programming in P...,0,0,0
18,一起来玩蹦女神！,0,0,0
24,Configuration control for production node depl...,0,0,0
27,Partly Cloudy today! With a high of 27°C and a...,0,0,0
34,xxxxxxxxxxx http://t.cn/h4NTd2,0,0,0


In [7]:
data_train[['text','repost','comments','likes']][data_train['is_noise']==1].head()

,text,repost,comments,likes
9,【励志】今天，我去买车了……（最近特别火的文章）http://t.cn/R2ug7bO,0,0,0
57,看到这样的公益项目，我忍不住要支持一把了，一起帮助“德义2015白血病患儿双胞胎姐妹张楚沅、...,0,0,0
128,我刚给@周冬雨 赠送了1朵鲜花，获得了2爱慕值，感觉自己萌萌哒！饭圈的亲们，快来送花应援吧！...,0,0,2
149,#马上有爱# 我获得了【爱的最强音】的微公益奖章。我会继续为公益摇旗呐喊！ http://t...,0,0,0
172,我刚换了“彩色天空”套装，好漂亮，你们都快来试试！ http://weibo.com/hom...,0,0,0


In [ ]:
data_train【